In [ ]:
from isimip_client.client import ISIMIPClient
client = ISIMIPClient()

In [ ]:
# select time series for a country
# run this subsequently to poll the status
path = 'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_tas_global_daily_2015_2020.nc'
response = client.select(path, country='nzl')
response

In [ ]:
# the same works for more then one file, e.g. a dataset, it just takes longer
paths = [
    'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_tas_global_daily_2015_2020.nc',
    'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_tas_global_daily_2021_2030.nc',
    'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_tas_global_daily_2031_2040.nc',
    'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_tas_global_daily_2041_2050.nc',
    'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_tas_global_daily_2051_2060.nc',
    'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_tas_global_daily_2061_2070.nc',
    'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_tas_global_daily_2071_2080.nc',
    'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_tas_global_daily_2081_2090.nc',
    'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_tas_global_daily_2091_2100.nc'
]
response = client.select(paths, country='nzl')
response

In [ ]:
# to select a time series for a bounding box in lat/lon use
path = 'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_tas_global_daily_2015_2020.nc'
response = client.select(path, bbox=[-23.43651, 23.43651, -180, 180])  # [south, north, west, east]
response

In [ ]:
# to select a time series for landonly (w/o antarctica):
path = 'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_tas_global_daily_2015_2020.nc'
response = client.select(path, landonly=True)
response

In [ ]:
# once the status is 'finished', get the url to download the result
client.download(response['file_url'], path='downloads', validate=False, extract=True)

In [ ]:
# this checking can be automated using poll=<time in seconds>
response = client.select(path, country='nzl', poll=2)
response

In [ ]:
# plot the time series
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from pathlib import Path

path = Path.cwd() / 'downloads' / 'gfdl-esm4_r1i1p1f1_w5e5_ssp370_tas_nzl_daily_2015_2020.csv'
df = pd.read_csv(path, names=['date', 'tas'], index_col='date', parse_dates=True, )
df.plot()
plt.gcf().set_size_inches(16, 8)

In [ ]:
# To select from a specific dataset, the following code can be used to first search the
# repository and then mask every file in the dataset. After starting the masking job 
# (on the server) it checks every 10 seconds, if the job is done and then downloads
# the file. Masking the files on the server can take a few minutes or even longer.

# get the dataset metadata from the ISIMIP repository
response = client.datasets(simulation_round='ISIMIP3b',
                           product='InputData',
                           climate_forcing='gfdl-esm4',
                           climate_scenario='ssp126',
                           climate_variable='tas',
                           time_step = 'daily')

dataset = response["results"][0]
paths = [file['path'] for file in dataset['files']]

# start/poll a masking job on the server (e.g. for China)
response = client.select(paths, country='chn', poll=10)

# download the file when it is ready
client.download(response['file_url'], path='downloads', validate=False, extract=True)

In [ ]:
# plot the time series
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

frames = []
for file in dataset['files']:
    file_name = file['name'].replace('global', 'chn').replace('.nc', '.csv')
    file_path = Path.cwd() / 'downloads' / file_name

    frames.append(pd.read_csv(file_path, names=['date', 'tas'], index_col='date', parse_dates=True))
    
df = pd.concat(frames)
df.plot()
plt.gcf().set_size_inches(20, 10)